## Team 6

### Team Members

- Dileep Ravindranath Holla
- Shaunak Prabhu

Link to Dataset: https://www.kaggle.com/unitednations/global-commodity-trade-statistics

In [1]:
import pandas as pd
import re
import numpy as np
from py2neo import Graph
import os 
import shutil 

### Read the raw CSV file

In [2]:
df = pd.read_csv('commodity_trade_statistics_data.csv')

/Users/dileepholla/PycharmProjects/DataGovernance/.virtualenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Data Cleaning

Replace all empty values or NaN with No Data across all columns

In [3]:
for i in df.columns:
    print('Checking field %s for Nulls' %i)
    df[i] = df[i].fillna('No Data')

Checking field country_or_area for Nulls
Checking field year for Nulls
Checking field comm_code for Nulls
Checking field commodity for Nulls
Checking field flow for Nulls
Checking field trade_usd for Nulls
Checking field weight_kg for Nulls
Checking field quantity_name for Nulls
Checking field quantity for Nulls
Checking field category for Nulls


### Unique ID

Add an unqiue Identifier for the Data

In [4]:
df["id"] = df.index + 1

### Write new CSV

In [5]:
df.to_csv('cleaned.csv', index = False)
print ('New CSV file created')

New CSV file created


### Move `CSV` file to the Neo4j `import` directory

In [6]:
destination = '/Users/dileepholla/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-e88658a5-5a5a-4681-9281-c6106b04fd76/import'
shutil.move('cleaned.csv', destination)
print ('File Moved')

File Moved


### Connect to Neo4j 

In [7]:
uri = "bolt://localhost:7687"
username = "neo4j"
pwd = "123456"

try:
    session = Graph(uri, auth=(username, pwd))
    print ('Connected to Neo4j!')
except:
    print ('Could not connect to Neo4j')

Connected to Neo4j!


### Create Constraints on Neo4j

In [8]:
constraints = [
    'CREATE CONSTRAINT com IF NOT EXISTS ON (com:Commodity) ASSERT com.id IS UNIQUE;',
    'CREATE CONSTRAINT flow IF NOT EXISTS ON (flo:Flow) ASSERT flo.flow IS UNIQUE;',
    'CREATE CONSTRAINT cat IF NOT EXISTS ON (cat:Category) ASSERT cat.category IS UNIQUE;',
    'CREATE CONSTRAINT country IF NOT EXISTS ON (cou:Country) ASSERT cou.country_or_area IS UNIQUE;',
    'CREATE CONSTRAINT year IF NOT EXISTS ON(yea:Year) ASSERT yea.year IS UNIQUE;',
    ]


In [9]:
for q in constraints:
    session.run(q)
    print ('%s query run sucessfully' %q)
    

CREATE CONSTRAINT com IF NOT EXISTS ON (com:Commodity) ASSERT com.id IS UNIQUE; query run sucessfully
CREATE CONSTRAINT flow IF NOT EXISTS ON (flo:Flow) ASSERT flo.flow IS UNIQUE; query run sucessfully
CREATE CONSTRAINT cat IF NOT EXISTS ON (cat:Category) ASSERT cat.category IS UNIQUE; query run sucessfully
CREATE CONSTRAINT country IF NOT EXISTS ON (cou:Country) ASSERT cou.country_or_area IS UNIQUE; query run sucessfully
CREATE CONSTRAINT year IF NOT EXISTS ON(yea:Year) ASSERT yea.year IS UNIQUE; query run sucessfully


### Load data to Neo4j

In [10]:
query = '''LOAD CSV WITH HEADERS FROM "file:///cleaned.csv" AS row
create (com:Commodity {CommodityName: row.commodity, CommodityCode: row.comm_code, Quantity: row.quantity, Weight: row.weight_kg, TradeValue: row.trade_usd, QuantityName: row.quantity_name})
MERGE (cat: Category {category: row.category})
MERGE (cou: Country {country: row.country_or_area})
MERGE (flo: Flow {flow: row.flow})
MERGE (yea: Year {year: row.year})
MERGE (com)-[:BELONGS_TO]->(cat)
MERGE (cou)-[:EXPORTS_IMPORTS_CAT]->(cat)
MERGE (cou)-[:REPORTS]->(yea)
MERGE (yea)-[:HAS_TYPE]->(flo)
MERGE (flo)-[:TRADES]-> (com)
MERGE (flo)-[:HAS_CATEGORIES]-> (cat);'''

In [11]:
try:
    session.run(query)
    print ('Loaded Data to Neo4j successfully!')
except Exception as e:
    print ('Could not load data. Error Details: %s' %e) 

Loaded Data to Neo4j successfully!
